# 对ctree文件夹的一些分析

## 代码分析环节

（这个section中请大家撰写对代码的分析，其中的关键点）

1. ctree中主要的文件为cminimax.cpp和cnode.cpp，前者主要用于EfficientZero paper的Backup过程中的公式10，也就是分别协助计算$\min_{(s,a)}Q(s,a)$和$\max_{(s,a)}Q(s,a)$，也就是一些statistic的信息。
1. cminimax.cpp中的`normalize`函数实现了EfficientZero paper中公式10的过程，注意$\epsilon$为0.01，`delta`小于该值的话， 则直接用$\epsilon$。
1. cnode.cpp涵盖了MCTS中的流程中Selection 、Expansion 、Backup。
    1. 从Selection开始：对应代码，可以先观察`cselect_child`函数。如下：
        1. `cucb_score`计算得到了paper中公式7对应的结果（在arg max之前）。但是它维护一个了`max_index_lst`，其特点就是只要是大于等于`max_score - epsilon`都会被加入该list，并在函数的结尾处从该list中随机sample一个action返回。
        1. 考虑`cucb_score`如何计算，见函数`cucb_score`：大的结构为`prior_score + value_score`，其中`value_score`为公式7中$Q(s,a)$项，`prior_score`为剩余的部分。`value_score`的计算较为复杂，其需要考虑到`child->visit_count`为0的情况。`parent_mean_q`对应于公式8的$\hat{Q}(s)$。如果$N(s,a)$不为0，那么，就是一般的$Q(s,a)$，计算方式也就是`true_reward + discount * child->value()`。
        1. 不管何种方式，$Q(s,a)$都需要被normalize，也就是公式10中的做法，相比MuZero，其多了0.01的`delta`的下限。值得注意的是，代码上对`value_score`做了clip到$[0,1]$的操作。
    1. 回到MCTS的Selection部分。因为这里实现的是所谓的batch MCTS，所以，真正的Selection部分为函数`cbatch_traverse`。如下对该函数进行简单的解释：
        1. batch MCTS涉及到`results.num`棵树。在for循环中有所体现。
        1. 对每棵树，从root开始，所以while循环之前`is_root`为1，之后就为0，在while中进行了设置。
        1. while循环的前提为`node->expanded()`，意思就是`node`还不是leaf。`mean_q`的含义为公式8中的$\hat{Q}(s)$，在cucb_score函数中会有用（`child->visit_count == 0`时）。
        1. while循环所找到的叶子为`node`，其的`parent`在`results.search_paths[i]`中下标为`results.search_paths[i].size() - 2`。
        1. `hidden_state_index_x/y_lst`存放的均为`int`类型数据，即下标值。
    1. 如下进入Expansion环节：
        1. 涉及到的函数有`prepare`，及`prepare_no_noise`，分别对应公式9中$\rho>0$或$\rho=0$的情况。注意$\rho=0$是有用的，见paper中所阐述的：However, we do not use any noise and set $\rho$ to 0 instead for those **non-root** node or during evaluations.
        1. 考虑较为复杂的`prepare`函数：
            1. 因为是batch MCTS，所以，for循环涉及`this->root_num`棵树。
            1. `expand`函数在对leaf进行展开。该函数的主要环节为一种数值稳定的计算softmax的方法。
                1. \begin{equation}\mathrm{softmax}(\mathbf{x})_i=\frac{\exp (x_i)}{\sum_j \exp (x_j)}\end{equation}
                1. $\mathbf{x}$更换为$\mathbf{z}=\mathbf{x}-\max_i x_i$
            1. `add_exploration_noise`就是公式9，即$(1-\rho)P(s,a)+\rho\mathcal{N}_\mathcal{D}(\xi)$，对于各个`child`好像可以采取不同的噪声，见`noises[a]`
    1. 最后考虑Backup部分：
        1. 该模块的起始点为`cbatch_back_propagate`函数，其考虑各个`results.nodes[i]`，从这个可以了解到`cbatch_traverse`函数中的最后一条语句`results.nodes.push_back(node);`，也就是说`cbatch_back_propagate`函数会先对各个leaf进行expand，然后，再从leaf到root进行backup。
        1. 实际实现backup功能的函数为`cback_propagate`。如下进行分析：
            1. for循环从下标`path_len - 1`开始考虑，该index对应leaf
            1. 分别对`node->value_sum`及`node->visit_count`进行更新。这个分别对应MuZero paper中的公式4。
            1. 接着获得`parent_value_prefix`，但如果是root，则该变量维持初始值0。否则考虑index `i - 1`，获得`parent->value_prefix`
            1. `bootstrap_value = true_reward + discount * bootstrap_value;`的理解如下：
                1. 其计算的为MuZero paper中的公式3，即$G^k$。但$G^k$和$G^{k-1}$的计算是有关的。
                1. \begin{equation*}G^k=\sum_{\tau=0}^{l-1-k}\gamma^\tau r_{k+1+\tau}+\gamma^{l-k}v^l\end{equation*}
                1. \begin{equation}
                \begin{split}
                G^{k-1}&=\sum_{\tau=0}^{l-1-(k-1)}\gamma^\tau r_{(k-1)+1+\tau}+\gamma^{l-(k-1)}v^l\\
                &=\sum_{0\le\tau\le l-1-(k-1)}\gamma^\tau r_{(k-1)+1+\tau}+\gamma\cdot\gamma^{l-k}v^l\\
                &=\sum_{0\le\tau+1\le l-1-(k-1)}\gamma^{\tau+1} r_{(k-1)+1+(\tau+1)}+\gamma\cdot\gamma^{l-k}v^l
                \\
                &=\sum_{-1\le\tau\le l-1-k}\gamma^{\tau+1} r_{k+1+\tau}+\gamma\cdot\gamma^{l-k}v^l\\
                &=r_k+\sum_{0\le\tau\le l-1-k}\gamma^{\tau+1} r_{k+1+\tau}+\gamma\cdot\gamma^{l-k}v^l\\
                &=r_k+\gamma G^k
                \end{split}
                \end{equation}
            1. 函数的最后确定`root`，以及对$\min_{(s,a)}Q(s,a)$和$\max_{(s,a)}Q(s,a)$等statistic进行清空，分别重置为`FLOAT_MAX`和`FLOAT_MIN`
            1. 进入到`update_tree_q`函数的调用，此函数分析如下：
                1. `update_tree_q`为基于stack的、非递归的depth-first search / DFS模块
                1. 惟一对MCTS有影响的部分在于`min_max_stats.update(qsa);`，其他的语句都是在实现DFS
    1. miscellaneous
        1. `get_trajectories`、`get_distributions`针对的是batch MCTS，可以直接考虑真正发挥作用的函数`get_trajectory`及`get_children_distribution`
        1. `get_trajectory`获取从root开始，沿`best_action`向下走。对于`node->best_action`来说，其的初始值为-1，且只在函数`cbatch_traverse`被改变，见`node->best_action = action;`，其中`action`由`cselect_child`返回，即EfficientZero中的公式7中的$\mathop{\arg\,\max}_a$
        1. `get_children_distribution`主要用于获得MCTS的策略，即EfficientZero中的公式11，即\begin{equation}\pi(s,a)=\frac{N(s,a)^{1/T}}{\sum_b N(s,b)^{1/T}}\end{equation}
    
## 讨论环节

（这个section中，请大家提出对repo中难理解部分的疑问，其他同学或老师看到了会进行解答）

1. this is an example question. posted by lezhang.thu@gmail.com

    cnode.cpp文件中，出现了大量的`is_reset`，不知道其语义是什么。
1. `expand`函数中，`ptr_node_pool->push_back(CNode(prior, action_num, ptr_node_pool));`，不知道为什么。`ptr_node_pool`又传入`CNode`的构造函数中，但`ptr_node_pool`在for循环中一直在更新。

    怀疑其和当前node获取children的列表有关，或许是当前node的children的list，怀疑。见函数`get_child`
1. `value`函数，本以为计算的是为MuZero paper中的公式4，但从代码中反复出现的语句`float qsa = true_reward + discount * node/child->value();`可以感觉到，$Q(s,a)$应该不是`value`函数的直接返回值。

    这一部分的逻辑没有理清楚。

# core文件夹下其他代码

## 代码分析环节

1. `train.py`文件中，首先观察`train`函数。
    1. `cpu_workers`、`gpu_workers`分别为`BatchWorker_CPU`和`BatchWorker_GPU`的实例。根据注释，这些都是**reanalyze workers**。对于**self-play workers**，其为`data_workers`，为`DataWorker`的实例。
    1. 先观察`selfplay_worker.py`文件。
1. `selfplay_worker.py`代码阅读：
    1. 考虑其中的`run`函数：
        1. `_get_max_entropy`函数：若`action_space`为$m$，则`ep`为\begin{equation}\sum_{0\le k<m} -\frac{1}{m}\log\frac{1}{m}\text{,}\end{equation}
        
        这是最大可能的entropy。
        1. `DataWorker`通过`self.replay_buffer`和外界交互，提供self play得到的训练数据。而`self.replay_buffer`获取数据，是在`free`函数中，其是从`self.trajectory_pool`中获取数据，并将后者清空。
        1. 而`self.trajectory_pool`拿到数据，是通过`put`函数。`put`函数在如下的3个地方被调用：
            1. `put_last_trajectory`函数中的`self.put((last_game_histories[i], last_game_priorities[i]))`
            1. `run`函数中有两处，形式均为：`self.put((game_histories[i], priorities)) `
        1. 从上述可以看出，self play产生的训练数据就是`game_histories[i]`。这主要是由于`put_last_trajectory`在`run`函数中被调用，相应的参数`last_game_histories`在`run`函数中也是来自于`game_histories`，见语句：`last_game_histories[i] = game_histories[i]`
        1. 现在的重点是`game_histories`是如何产生的。
            1. 调用`MCTS(self.config).search`之后，可以获得`roots_distributions = roots.get_distributions()`及`roots_values = roots.get_values()`
            1. 根据`roots_distributions`及`roots_values`，考虑`start_training`来决定是否采取random policy，最终得到`distributions`和`value`
            1. 有了`distributions`，其会调用`core.utils`中的`select_action`函数，即计算\begin{equation}\pi(s,a)=\frac{N(s,a)^{1/T}}{\sum_b N(s,b)^{1/T}}\text{,}\end{equation}

            据paper中所写，$T$的initial为1.0，在50\% training时，为0.5；在75\% training时，为0.25。这些都在`config/atari/__init__.py`有所实现（但貌似`num_moves`在计算时没有用处）。`select_action`函数最终会返回应该执行的`action`
            1. `action`和环境交互之后并进行处理，得到`obs`和`clip_reward`，这些将会被存储到`game_histories[i]`中。
            1. 在`run`函数中，`game_histories`为python的list，其的length为`env_nums`，其中每一个都为`GameHistory`的实例。并且，还进行了`game_histories[i].init(stack_obs_windows[i])`，其中，`stack_obs_windows[i]`来自于`init_obses[i]`，而后者来自于`env.reset()`。

            相同的模式在`run`函数的之后会再次出现，针对于`done[i]`为`True`时。
        1. 对`run`函数的、大的结构的分析如下：
        ```
            while True:
                # training finished
                if trained_steps >= self.config.training_steps + self.config.last_steps:
                    break
                # play games until max moves
                while not dones.all() and (step_counter <=
                                           self.config.max_moves):
                    for i in range(env_nums):
                        # reset env if finished
                        if dones[i]:                                                               
                for i in range(env_nums):
        ```

        注意，`while True:`中只有一条`break`语句。
1. 针对于`selfplay_worker.py`的`run`函数：
    * 调用`MCTS(self.config).search`之前，`roots`都是最新实例化的。这一点和围棋中的做法不同，后者不是对之前的MCTS tree进行整体丢弃，而这里是整体丢弃，重新实例化`roots`。
    * 同样在调用`MCTS(self.config).search`之前，会调用`roots.prepare`，根据“对ctree文件夹的一些分析”中所阐述的内容，`prepare`和Expansion环节有关。
      
      仔细观察可以发现，`cnode.cpp`中的`prepare`和`prepare_no_noise`都只是和roots有关，并不涉及一般的叶子节点。后者的expand，在`cbatch_back_propagate`中已经处理，其中就有调用`expand`函数，而其实`prepare`和`prepare_no_noise`之流，也是调用`expand`函数，只不过是针对roots而已。

      另外，从`mcts.py`中也可以看出，和MCTS的3个环节有关的调用只有：`tree.batch_traverse`和`tree.batch_back_propagate`，可以观察到Expansion环节（针对于叶子节点）已经在`tree.batch_back_propagate`中被处理了。
    * MCTS给self play提供的、最关键的信息到底有哪些？

      ```
        # store data
        game_histories[i].store_search_stats(
            distributions, value)
        game_histories[i].append(action, obs, clip_reward)      
      ```
1. 针对于`mcts.py`的分析：
    * efficientzero和muzero不同，后者的dynamics会提供所需要的reward（当然是dynamics自己估计的），efficientzero采取的却为value prefix的形式，是有专门的*value prefix* prediction network: $r_t,h_{t+1}=\mathcal{R}(\hat{s}_{t+1},h_t)$来处理
    * 在`mcts.py`，因为`cnode.cpp`为c++代码，不可能具备计算value prefix的能力，所以，是通过传入`value_prefix_pool`来实现的。
    * 在efficientzero中，其是有value prediction network的。其的形式见paper的Appendix A.1中的eq. 7，形式为$v_t=\mathcal{V}(s_t)$。这个值的用处在于计算：

      \begin{equation*}
      G^k=\sum_{\tau=0}^{l-1-k}\gamma^\tau r_{k+1+\tau}+\gamma^{l-k}v^l
      \end{equation*}

      见上式中的$v^l$。

      在`mcts.py`中，这一值为`value_pool`，并同样传入了`tree.batch_back_propagate`。

      `value_pool`和计算上式中的$v^l$之间的关系，见`ctree/cnode.cpp`中的`cback_propagate`函数，其中有：`float bootstrap_value = value;`
    * `reward_hidden_h_pool`是用来计算value prefix的，其为LSTM中的hidden state。`mcts.py`中有：`reward_hidden_h_pool = [reward_hidden_roots[1]]`，其上的注释中写明“`# 1 x batch x 64`”，注意这是说`reward_hidden_roots[1]`的shape为1 x batch x 64，也因为如此，后续对其index时，为：

      ```
        hidden_states_h_reward.append(
            reward_hidden_h_pool[ix][0][iy])
      ```

      注意索引时为`[ix][0][iy]`，中间为0，这是因为1 x batch x 64中的1。

    * `rewards_hidden_roots`来自`selfplay_worker.py`文件中`model.initial_inference`。后者进行的工作见`core/model.py`，观察可知其进行的为`# zero initialization for reward (value prefix) hidden states`。这一点很重要，因为这将和reset相关联（reset不仅出现在`mcts.py`中，也会出现在`cnode.cpp`中）。
      ```
        # reset 0
        # reset the hidden states in LSTM every horizon steps in search
        # only need to predict the value prefix in a range (eg: s0 -> s5)
      ```

      `rewards_hidden_roots`带来`reward_hidden_h_pool`，后者的作用是计算value prefix。刚开始时，来自roots，也就是`rewards_hidden_roots[1]`。后续因为expand的肯定是叶子节点，所以其来自于上一步展开的叶子节点。

      所以，我们需要存储上一步中展开的叶子节点的hidden state，在代码中表现为：`reward_hidden_h_pool.append(reward_hidden_nodes[1])`。同时有`hidden_state_index_x += 1`，这是因为之后关注的是$h_{t+1}$，而不是上一步的$h_t$。这一点请考虑$r_t,h_{t+1}=\mathcal{R}(\hat{s}_{t+1},h_t)$。

      最后考虑reset的过程。reset的过程就是将$h_t$置为0，就和`model.initial_inference`时一样。`mcts.py`中是这样做的：

      ```
        # reset 0
        # reset the hidden states in LSTM every horizon steps in search
        # only need to predict the value prefix in a range (eg: s0 -> s5)
        assert horizons > 0
        reset_idx = (np.array(search_lens) % horizons == 0)
        assert len(reset_idx) == num
        reward_hidden_nodes[0][:, reset_idx, :] = 0
        reward_hidden_nodes[1][:, reset_idx, :] = 0
        is_reset_lst = reset_idx.astype(np.int32).tolist()
      ```

      只有那些在MCTS的select环节，从root到某个被select的叶子的路径长度为5的倍数时，计算value prefix的$h_t$才会被reset为0。有关reset的机制，需要联系`ctree/cnode.cpp`的`cbatch_traverse`函数来理解。

      对value prefix的计算过程的理解，可以帮助我们明白为什么在`cbatch_traverse`中需要返回`last_action`，而不是`action`，也就是说只需要记录“导致”最后要expand的叶子节点的`last_action`，因为$h_t$从parent而来，这不需要记录完整的、从root到leaf的路径。
1. 对`reanalyze_worker.py`的阅读。了解其，需要先阅读paper中的A.4 Training Details，其中的Pipeline，了解其中的`BatchWorker_CPU`和`BatchWorker_GPU`都需要发挥什么样的作用。
    * 首先观察`BatchWorker_CPU`。在其的`run`函数中，其首先调用`replay_buffer.prepare_batch_context`，在调用其自己的`make_batch`函数。因此，我们首先需要了解`replay_buffer.py`中的机制。
    * 对`replay_buffer.py`的阅读：
        * 首先，replay buffer在`selfplay_worker.py`中有所交互，见后者中的`free`函数（其调用replay buffer中的`save_pools`函数）。而后者中的`trajectory_pool`做了什么？数据进入`trajectory_pool`主要是通过`selfplay_worker.py`中的`put`函数。该函数的使用形式主要是`self.put((game_histories[i], priorities))`等。而其中的`game_histories[i]`中的信息主要有：

          ```
            # store data
            game_histories[i].store_search_stats(                                         
                distributions, value)                                                     
            game_histories[i].append(action, obs, clip_reward)     
          ```
        * 如下先理解`replay_buffer.py`中的`save_pools`函数：
            * `save_pools`函数的功能主要由`save_game`完成。
            * 若不考虑`save_game`中的`priorities`，则`save_game`中的`game`参数相当于上述所说的`game_histories[i]`。
            * `save_game`中针对`game`所做出的重要操作为：`self.buffer.append(game)`，即`game_histories[i]`作为一个整体存储到buffer中。
            * `self.game_look_up`的作用TODO。需要注意的是，在`selfplay_worker.py`中，只有`if dones[i]:`时，才会有`self.put((game_histories[i], priorities))`。这说明`game_histories[i]`存储的不会是single transition，而可能是多个连续的transitions。`self.game_look_up`是通过`(self.base_idx + len(self.buffer) - 1, step_pos)`这样的二维index来索引所有的transitions吗，因为`step_pos`被其穷举，所以，`self.game_look_up`是通过一维的方式来索引？
        * `get_game`函数：因为是获取`game`，所以，是获取`game`的整体，而不是其中某single transition，因此，获取其在`self.buffer`中的下标就可以了。和`(self.base_idx + len(self.buffer) - 1, step_pos)`是相逆的过程。
        * `update_priorities`函数：和优先级有关的函数，TODO。
        * `remove_to_fit`和`_remove`，前者依赖后者，逐一查看一下。在`remove_to_fit`中，是根据`self.transition_top`来对`self.buffer`进行部分清空（见`_remove`中的`del self.buffer[:num_excess_games]`），不同的`game`贡献的transitions的长度不同，所以，在`remove_to_fit`中，从下标0的`game`开始考虑，逐一累加该`game`贡献的transitions的长度，直到若删除这些`game`，总的transitions的个数`<= self.transition_top * self.keep_ratio`。从`_remove`函数中的`self.base_idx += num_excess_games`可以感觉到`self.base_idx`是一个**虚拟的时间index**，这也是其初值为0后，惟一的赋值操作。
        * `self.priorities`和`self.buffer`是不一样的，前者采取的是`self.priorities = np.concatenate((self.priorities, priorities))`。这表明了`self.priorities`是拆开来之后进行拼接的。
        * `prepare_batch_context`函数。首先其中的`make_time`可以忽略。和优先级有关的`weights_lst`先不考虑。对于关键的`game_lst`和`game_pos_lst`，首先该函数中的如下做法：

          ```
            game_id, game_pos = self.game_look_up[idx]
            game_id -= self.base_idx
            game = self.buffer[game_id]
          ```

          和`get_game`函数中是一致的。`game_pos`相当于是`game`中哪一个transition的下标。
    * 再次回到`reanalyze_worker.py`的`BatchWorker_CPU`。
        * 考虑其中的`make_batch`函数。
            * `make_time_lst`不重要，可不考虑；`weights_lst`涉及到优先级，TODO。
            * `_actions`肯定要取到`self.config.num_unroll_steps`这么多个，不足的补随机动作，见：

              ```
                _actions += [
                    np.random.randint(0, game.action_space_size)
                    for _ in range(self.config.num_unroll_steps - len(_actions))
                ]
              ```

              相应的，`_mask`则表示相应的动作是否是实际取出来的（从`game.actions`中）：为1.表示是的；为0.表示不是的。

              `obs_lst`和`action_lst`很相似，前者特别用`padding=True`来表示类似的含义。
            * 接下来确定reanalyze的部分，注意value是百分百reanalyze，可见注释，也可见paper中A.4 Training Details的Reanalyze部分，"Besides, we reanalyze the policy among
99% of the data and reanalyze the value among 100% data."
            * `obs_lst`被`prepare_observation_lst`处理了，后者进行了`[B, S, W, H, C] -> [B, S x C, W, H]`，含义为`batch, stack num, width, height, channel`
        * 考虑`_prepare_policy_re_context`函数
            * 重要的是`policy_obs_lst`，以及`policy_mask`
            * `policy_mask`较容易理解，1或者0，表示是否越过了`traj_len = len(game)`
            * 越过了`traj_len`，则附上`zero_obs`；否则附上真实的`game_obs[beg_index:end_index]`
            * `game_obs`的下标从0开始，其含有`config.num_unroll_steps`个observations，但其中的每一个都涉及到`config.stacked_observations`，所以，总的长度为`config.num_unroll_steps * config.stacked_observations`
            * 如下的代码：

            ```
            for current_index in range(
                    state_index,
                    state_index + config.num_unroll_steps + 1):
            ```

            略画蛇添足，其实，`current_index`应该从0开始，这样后续就不用再进行`beg_index = current_index - state_index`
## 讨论环节


1. `_prepare_policy_non_re_context`的代码是`_prepare_policy_re_context`的一个子集，所以，可以只关注后者。

